In [11]:
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import copy

In [12]:
# load sptial anndata object
lung_A = ad.read_h5ad('../data/OneDrive_1_04-11-2024/adata_LNEN071-IARC-A_scanpy_06052024.h5ad')

lung_B = ad.read_h5ad('../data/OneDrive_1_04-11-2024/adata_LNEN084-IARC-B_scanpy_06052024.h5ad')

lung_C= ad.read_h5ad('../data/OneDrive_1_04-11-2024/adata_LNEN107-IARC-C_scanpy_06052024.h5ad')

lung_D= ad.read_h5ad('../data/OneDrive_1_04-11-2024/adata_LNEN206-IARC-D_scanpy_06052024.h5ad')

In [3]:
# load IRIS annotation by spatial domain
iris_annot = pd.read_csv('../data/iris_annotations_NEW.csv')
iris_annot.head()

,Slice,spotName,x,y,IRIS_domain
0,LNEN071-IARC-A,AAACAAGTATCTCCCA-1,1125,745,9
1,LNEN071-IARC-A,AAACACCAATAACTGC-1,819,2342,5
2,LNEN071-IARC-A,AAACAGAGCGACTCCT-1,2331,902,1
3,LNEN071-IARC-A,AAACAGGGTCTATATT-1,1220,2459,15
4,LNEN071-IARC-A,AAACAGTGTTCCTGGG-1,351,1878,4


In [4]:
def process_iris(iris_annot, slice):
    """Filter annotations to use only one slice."""
    
    iris_annot_slice = iris_annot.copy()
    iris_annot_slice = iris_annot_slice[iris_annot_slice["Slice"] == slice]
    
    iris_annot_slice.set_index("spotName", inplace = True)
    iris_annot_slice.index.name = None

    iris_annot_slice = iris_annot_slice[["Slice", "x", "y", "IRIS_domain"]]

    iris_annot_slice["IRIS_domain"] = iris_annot_slice["IRIS_domain"].astype("string")

    return iris_annot_slice

In [7]:
iris_annot_slice_A = process_iris(iris_annot, "LNEN071-IARC-A")
print(iris_annot_slice_A.head())

iris_annot_slice_B = process_iris(iris_annot, "LNEN084-IARC-B")
print(iris_annot_slice_B.head())

iris_annot_slice_C = process_iris(iris_annot, "LNEN107-IARC-C")
print(iris_annot_slice_C.head())

iris_annot_slice_D = process_iris(iris_annot, "LNEN206-IARC-D")
print(iris_annot_slice_D.head())

                             Slice     x     y IRIS_domain
AAACAAGTATCTCCCA-1  LNEN071-IARC-A  1125   745           9
AAACACCAATAACTGC-1  LNEN071-IARC-A   819  2342           5
AAACAGAGCGACTCCT-1  LNEN071-IARC-A  2331   902           1
AAACAGGGTCTATATT-1  LNEN071-IARC-A  1220  2459          15
AAACAGTGTTCCTGGG-1  LNEN071-IARC-A   351  1878           4
                             Slice     x     y IRIS_domain
AAACAAGTATCTCCCA-1  LNEN084-IARC-B  1125   772           7
AAACAATCTACTAGCA-1  LNEN084-IARC-B  2696  1913           7
AAACACCAATAACTGC-1  LNEN084-IARC-B   818  2369          11
AAACAGAGCGACTCCT-1  LNEN084-IARC-B  2331   930           7
AAACAGGGTCTATATT-1  LNEN084-IARC-B  1220  2486          11
                             Slice     x     y IRIS_domain
AAACAAGTATCTCCCA-1  LNEN107-IARC-C  1111   741          12
AAACAGAGCGACTCCT-1  LNEN107-IARC-C  2317   898          18
AAACAGCTTTCAGAAG-1  LNEN107-IARC-C  1340  2532          19
AAACAGGGTCTATATT-1  LNEN107-IARC-C  1207  2455          

In [8]:
def add_iris_annot(adata, iris_annot_slice):
    """Add iris metadata to the adata.obs. Create new anndata object with only necessary metadata in .obs."""
    adata_annot_obs = adata.obs.join(iris_annot_slice)
    adata_obs = copy.deepcopy(adata_annot_obs)
    adata_obs = adata_obs[["in_tissue", "Slice", "x", "y", "IRIS_domain"]]
    
    adata_annot = adata.copy()
    adata_annot.obs = adata_obs

    return adata_annot

In [9]:
lung_A_annot = add_iris_annot(lung_A, iris_annot_slice_A)
lung_B_annot = add_iris_annot(lung_B, iris_annot_slice_B)
lung_C_annot = add_iris_annot(lung_C, iris_annot_slice_C)
lung_D_annot = add_iris_annot(lung_D, iris_annot_slice_D)

In [10]:
# save the annotated data
lung_A_annot.write("../data/annotated_slice1_NEW.h5ad", compression="gzip")
lung_B_annot.write("../data/annotated_slice2_NEW.h5ad", compression="gzip")
lung_C_annot.write("../data/annotated_slice3_NEW.h5ad", compression="gzip")
lung_D_annot.write("../data/annotated_slice4_NEW.h5ad", compression="gzip")